連線database

In [90]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=1000)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# 可自由調整select的區間
# today_str = '20240813'
today_str = '20241231'
one_year_ago_str = '20210101'

# SQL 查詢 加權指數
query_daily_tx = f"""
SELECT *
FROM daily_tx
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 加權指數
query_daily_updown = f"""
SELECT *
FROM daily_updown
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 加權指數
query_tif_investors = f"""
SELECT *
FROM daily_tif_investors
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 加權指數
query_pc_ratio = f"""
SELECT *
FROM daily_pc_ratio
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
tif_investors_df_raw = pd.read_sql(query_tif_investors, engine)
# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
updown_df_raw = pd.read_sql(query_daily_updown, engine)
# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
tx_df_raw = pd.read_sql(query_daily_tx, engine)

pc_ratio_raw = pd.read_sql(query_pc_ratio, engine)

stock_df_raw = pd.read_sql(query_stock, engine)
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])

https://www.wantgoo.com/index/vixtwn

波動率可以用這邊的 臺指選擇權波動率指數(VIXTWN)

In [1]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S0, K, T, r, sigma):
    """
    計算台指期選擇權的 Call 價格（使用 Black-Scholes 公式）
    
    參數:
        S0: 現貨價格
        K: 履約價格
        T: 距到期時間（年）
        r: 無風險利率
        sigma: 波動率
    
    返回:
        Call 選擇權的理論價格
    """
    # 計算 d1 和 d2
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Black-Scholes Call 價格
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    return call_price , put_price

# 測試
S0 = 22461  # 現貨價格
K = 22250   # 履約價格
T = 7 / 365  # 距到期時間（30天）
r = 0.02     # 無風險利率（年化 2%）
sigma = 0.2143 # 波動率（15%）

call_price, put_price = black_scholes_call(S0, K, T, r, sigma)
print(f"Call 選擇權價格: {call_price:.2f}")
print(f"Put 選擇權價格: {put_price:.2f}")


Call 選擇權價格: 388.75
Put 選擇權價格: 169.21
